# Diversity of Wikipedia article references

## Mining massive databases course final project

### Problem motivation: 

The diversity of resources and content matters. When people are obtaining new knowledge, they don't want to be fooled with fake news or believe in information without proof from other authorized source. A lot of Wikipedia articles have a problem with poor filled or unreferenced sources of information, so, in the era of post-truth, there is a huge need for automatic detection of articles with poor sources distribution (or without sources as well). <br> <br>
    Our solution can help editors as well, because they will have possibility to concentrate on the truly non-filled arctiles. Also, the main improvement for editors and readers that the differences in the same article across languages can be detected and fixed. 

### Problem statement: 

Estimate the quality of the article, based on the references, in an unsupervised way. Unite and check the results of our modeling with results from ORES model. 

### Work pipeline: 

### Data processing: 

* Download wikipedia XML dumps: <br> <br>
 * We are using page article multistream dumps. To get faster development loops so far we worked with a single dump, next we will run the full pipeline on the whole wikipedia data. <br> <br>
* Parse XML to CSV using streaming XML parser: <br> <br>
 * We are using lxml and handwritten parser that goes through the file tag by tag and parses articles and meta information and article and last revision. The data we are fetching includes article text, title, revision author, revision comment and timestamp. <br> <br>
* Fetch ORES assessments: <br> <br>
 * Here we use mwapi and ORES web service to get article scores <br> <br>
* Inspect internal structure of text: <br> <br>
 * Wikipedia articles have its own syntax for declaring blocks inside
   article: source

In [117]:
# add smth else

### Feature Engineering

<font size=4> Were built features that reflect diversity of text, sources and links: <br> <br>
* Internal, external references count <br> <br>
* Average number of references per block of text (Number of references / Number of paragraphs) <br> <br>
* Citations count (Journals, Books, Web, News) <br> <br>
* Number of images, files, etc in the articles <br> <br>
* Number of non-approved references (“citation needed”) <br> <br>
* Headings count (different levels) <br> <br>
* ORES features 

In [116]:
# download here the features dataframe and demonstrate features

### Modeling:

In [114]:
# Silhouette coefficient theory (more detailed, mb outputs from plots post also here) + clusteing

### Evaluation: 

In [115]:
# call there python ./test.py -t "Principal component analysis"
# and test smth else

### Future work: 

<font size=4> 
* Create supervised machine learning model with ORES as labels and our features as inputs. The resulted model should be transferred to the other languages that didn’t support by ORES (like Ukrainian) <br> <br> 
* Test the PySpark MLP / Random Forest / Gradient boosting and get the features importance (visualize the pluses and minuses of the articles references) <br> <br>
* Fix clustering with ORES features <br>